# **train.ipynb**

본 파일에서는, **전처리한 네이버 영화 리뷰 데이터셋을 활용하여 우리가 생성한 BertModel을 학습**시켜볼 것이다. 이후, **최종 성능(테스트 정확도)까지 출력**해볼 것이다.



연습 문제를 시작하기에 앞서, import_ipynb를 설치한 후 import 하여 train.ipynb에서 필요한 preproc.ipynb, model.ipynb 내 함수를 호출한다.  
또한, 필요한 라이브러리를 호출한다.

In [7]:
import import_ipynb

from Preproc import preproc
from model import get_model, get_model_with_params, BertModelInitialization
import random
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm
import time

ImportError: cannot import name 'preproc' from 'Preproc' (Preproc.ipynb)

모델의 예측 정확도를 산출하는 함수인 **accuracy**를 정의하자. 해당 함수는 학습한 모델의 validation 점수와 test의 결과를 계산할 때 사용된다.


In [ ]:
# 정확도 계산 함수
def accuracy(preds, labels):
    f_pred = np.argmax(preds, axis=1).flatten()
    f_labels = labels.flatten()
    return np.sum(f_pred == f_labels) / len(f_labels)

**잠깐 ✔ 랜덤시드 고정이란 무엇인가?**
> 학습된 모델의 결과를 동일하게 재현(Reproduction)하는 것은 여러가지 상황에서 팔요하다.  
> 모델을 돌릴 때마다 결과가 달라지지 않도록 고정하는 것이다.  

- 수상자가 되어 코드의 정합성을 검증 받게 될 경우,

- 경진대회 참가 도중 팀을 이루어 결과를 공유해야 되는 경우,

- 논문을 작성하여 그 결과를 Reproduction 해야하는 경우 등 여러 상황에서 필요하다.  

- 본 과제 역시, (1) preproc.ipynb 내 섹터 별 자동 점수 반환 및 (2) 최종 평가 과정에서 혼동을 방지하기 위하여 랜덤시드를 고정해야 한다. 주어진 2022 시드 값을 절대 수정하지 않도록 하자.

참고 자료:
https://dacon.io/codeshare/2363
https://pytorch.org/docs/stable/notes/randomness.html


In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 2022

In [ ]:
# 랜덤하게 데이터를 추출하기 위한 seed 값 설정
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

학습에 활용될 데이터셋 및 토크나이저를 지정하자. 이후, 데이터셋을 전처리하여 train, validation, test 각각의 데이터로더에 입력하자.


In [ ]:
from tokenization import KoBertTokenizer

# 전체 데이터를 불러오자.
whole_dataset = pd.read_csv('ratings.txt', delimiter="\t")

# KoBERTTokenizer를 불러오자.
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")

train_dataloader, validation_dataloader, test_dataloader = preproc(tokenizer, whole_dataset)

BertModel을 생성하여 GPU 혹은 CPU에 등록하자.
- 이때 BertModelInitialization()를 실행할 경우 기존 Device에 등록된 BertModel은 초기화되니, 한 번만 실행한 이후로는 사용하지 않도록 유의하여 사용해야 한다.
- 디바이스를 설정하자.
- 본격적인 학습에 앞서 train에 대한 model, 옵티마이저, 스케줄러, 에폭을 지정하고, 모델의 그래디언트를 초기화하자.

In [ ]:
# 기존 Device에 등록된 BertModel은 초기화되니, 유의하여 사용할 것.
# 한 번만 실행하고, 그 이후로는 사용하지 않도록 조심!
BertModelInitialization()

In [ ]:
# GPU 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model, optimizer, scheduler, epochs = get_model_with_params(len(train_dataloader), device)

# 그래디언트 초기화
model.zero_grad()

**train 및 validation**  
본격적으로 학습을 진행해보자. epoch 만큼 학습 loop를 반복할 것이다.
- 우리가 생성한 model에, 배치 데이터에 대한 input_ids, attention_mask, labels 변수를 입력하여 순전파를 진행할 것이다.
- 이후 역전파 과정을 통해 매개변수가 조절되며 학습이 이루어진다.
- 한 차례 학습이 이루어질 때마다 average training loss 및 validation 정확도를 출력할 것이다.
- 학습이 완료된 모델을 특정 경로(PATH)에 저장할 것이다.


In [ ]:
# 에폭만큼 반복
for epoch_i in range(epochs):
    print("")
    print('========{:}번째 Epoch / 전체 {:}회 ========'.format(epoch_i + 1, epochs))
    print('훈련 중')

    total_loss = 0 # 로스 초기화
    sum_loss = 0
    model.train()  # 훈련모드로 변경

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(tqdm(train_dataloader)):

        if step % 50 == 0:
          print("{}번째 까지의 평균 loss : {}".format(step, sum_loss/50))
          sum_loss = 0

        batch = tuple(t.to(device) for t in batch)   # 배치를 GPU에 넣음
        b_input_ids, b_input_mask, b_labels = batch  # 배치에서 데이터 추출

        # model.ipynb에서 model 함수를 정의할 때 BertForSequenceClassification를 활용하였다.
        # 여기서 BertForSequenceClassification는 input_ids, attention_mask, labels 변수를 입력받는 'forward' 함수를 내장한다.
        # forward 함수는 forward(self, input_ids, attention_mask, token_type_ids, position_ids, head_mask, inputs_embeds, labels, output_attentions, output_hidden_states)와 같은 함수 파라미터를 갖는다.
        # model 함수를 통해 forward를 수행하기 위해 우리가 입력해야 하는 변수는 input_ids, attention_mask, labels 이다.
        # 위의 코드에서 정의한 배치 데이터를 model 함수에 입력하여, 배치에 대한 Forward를 수행해보자.

        ##여기에 코드 작성
        outputs = ------------------------------------

        loss = outputs[0]
        total_loss += loss.item() # 총 로스 계산
        sum_loss += loss.item()

        loss.backward() # Backward 수행으로 그래디언트 계산
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # 그래디언트 클리핑
        optimizer.step() # 그래디언트를 통해 가중치 파라미터 업데이트
        scheduler.step()  # 스케줄러로 학습률 감소
        model.zero_grad() # 그래디언트 초기화

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))

    #### 검증 ####

    print("")
    print("검증 중")

    model.eval()

    # 변수 초기화
    eval_accuracy = 0
    nb_eval_steps = 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)

        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # 그래디언트 계산 안함
        with torch.no_grad():
            # Forward 수행
            outputs = ------------------------------------

        # 결과 값 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))

# 학습된 모델을 해당 PATH에 저장
PATH = "model.pt"
torch.save(model.state_dict(), PATH)

print("")
print("Training complete!")

**test**  
학습된 모델에 test용 데이터를 입력하여 test 결과를 출력하자.
- test용 데이터로더를 활용하여 배치 데이터에 대한 input_ids, mask를 model에 입력하여 순전파를 실행하고, 실행 결과를 outputs에 저장하자.
- 이때 test는 '학습'이 목적이 아니므로 그래디언트를 계산하지 않도록 한다.
- outputs와 실제 정답인 label_ids를 비교하여 모델의 최종 test accuracy를 확인하자. 모델이 88%의 정확도를 넘기는가?


In [ ]:
print("")
print("테스트 중")

model.eval()

# 변수 초기화
eval_accuracy = 0
nb_eval_steps = 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for batch in test_dataloader:
  # 배치를 GPU에 넣음
  batch = tuple(t.to(device) for t in batch)

  # 배치에서 데이터 추출
  b_input_ids, b_input_mask, b_labels = batch

  # 그래디언트 계산 안함
  with torch.no_grad():
      # Forward 수행
      outputs = ------------------------------------

  # 결과 값 구함
  logits = outputs[0]

  # CPU로 데이터 이동
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  # 출력 로짓과 라벨을 비교하여 정확도 계산
  tmp_eval_accuracy = accuracy(logits, label_ids)
  eval_accuracy += tmp_eval_accuracy
  nb_eval_steps += 1

print("")
print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))